# **Lecture 08: 딥러닝의 기본 개념**

>**Brain & Neuron**   
&rarr; 복잡하다<br><br>
<img src="https://www.simplypsychology.org/wp-content/uploads/Neuron.jpg" height=300><br><br>
- axon from antoher nueron &rarr; $x_{0}$  
- synapse &rarr; $w_{0}$  
- cell body &rarr; $\sum({w_{i}x_{i} +b_{i}})$ and  **activation function**  
- output axon &rarr; $f(\sum({w_{i}x_{i} +b_{i}}))$  and **sigmoid function**  

>**Linear separation**  
- AND / OR can linearly separable
- XOR can not linearly separable<br><br>
<img src="https://b2633864.smushcdn.com/2633864/wp-content/uploads/2021/04/bitwise_datasets-1024x365.png?lossy=2&strip=1&webp=1">  

>**Perceptron**  
- Made by Morkin Minsky  
- MLP(Multiple perceptron ) can help calculate XOR, but we can not find the weigth of MLP  

>**Backprogation**  
- weigth 값 수정을 앞에서 차례대로 바꿔서 하는 것이 아니고, 결과 값에 error가 생기면 wieght에 수정을 해줘서 찾는 방식이다  

>**Convolutional Neural Network, CNN**  
- 부분적으로 문자나 사진을 나눠서 인식하여 output을 사용하여 나타내는 것  

>**Big Problem**  
- Backpropagation just did not work well for normal neural nets  
- other risng machine learning Algrithm such as SUM, RandomForest is better than CNN  

>**Back Through**  
- If weights are initialized in a clear way  
- Rebranding to **Deep Learning**  

#**Lecture 09: XOR 문제 딥러닝으로 풀기**

> **XOR**  
- One logistic Regression is unit can not seperate  
- Exclusive or gate  
- $W_{1}x_{1} + b_{1}, W_{1}= \left[\begin{matrix} 5 \\ 5 \end{matrix}\right], b_{1} = -8$<br><br>
- $W_{2}x_{2} + b_{2}, W_{2}= \left[\begin{matrix} -7 \\ -7 \end{matrix}\right], b_{2} = 3$<br><br>
- $W_{3}x_{3} + b_{3}, W_{3}= \left[\begin{matrix} -11 \\ -11 \end{matrix}\right], b_{3} = 6$<br><br>
- Can create XOR gate  

> **Forward Propagation**<br><br>
<img src="https://www.enjoyalgorithms.com/static/forward-propagation-in-neural-networks-6df57409c0c35d7b25909737eb81b3ec.png" heigth=100 width=500><br><br>
- 여러가지 gate를 만들어서 여러층으로 만든다  

> **Neural Network**  
- Forward propagation에서 여러층으로 된 Input layer를 행렬 형태로 변환 시킨 네트워크  
- $k(X) = sigmoid(XW_{1} + B_{1})$  
- $\overline{Y} = H(X) = sigmoid(k(X)W_{2} + b_{2})$  

> **Backpropagation(chain rule)**  
- How can we learn W and B from data?  
- 중간에 어떤 데이터들이 있더라도 처음 값과 마지막 값만 있으면 chain rule을 이용하여 미분할 수 있다


# **Lab 09-1: Neural Net for XOR**

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

>**Neural Network를 사용하지 않은 코드**  
&rarr; 각각 sigmoid와 Cost function을 직접 정의 해서 3개의 function을 정의

In [ ]:
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

#Data for feature and label
def preprocess_data(features, labels):
    features = tf.cast(features, tf.float32)
    labels = tf.cast(labels, tf.float32)
    return features, labels

#Weight and Bias for hypothesis
W = tf.Variable(tf.zeros((2,1)), name='weight')
b = tf.Variable(tf.zeros((1,)), name='bias')
print("W = {}, B = {}".format(W.numpy(), b.numpy()))

#Hypothesis for sigmoid
def logistic_regression(features):
    hypothesis  = tf.divide(1., 1. + tf.exp(tf.matmul(features, W) + b))
    return hypothesis

#Cost Function for loss
def loss_fn(hypothesis, features, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(logistic_regression(features)) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

#Accuracy Fuction
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

#Gradient Descent and Fitting
def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(logistic_regression(features),features,labels)
    return tape.gradient(loss_value, [W,b])

EPOCHS = 1001

for step in range(EPOCHS):
    for features, labels  in dataset:
        features, labels = preprocess_data(features, labels)
        grads = grad(logistic_regression(features), features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
        if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(logistic_regression(features),features,labels)))
print("W = {}, B = {}".format(W.numpy(), b.numpy()))
x_data, y_data = preprocess_data(x_data, y_data)
test_acc = accuracy_fn(logistic_regression(x_data),y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))

> **Nueral network를 사용한 코드**  
&rarr; 3개의 Layer를 하나의 matrix로 만든 다음 행렬 연산을 통해 학습을 시작한다

In [ ]:
#Load data Set
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

def preprocess_data(features, labels):
    features = tf.cast(features, tf.float32)
    labels = tf.cast(labels, tf.float32)
    return features, labels

#Setting Weight and Bias
W1 = tf.Variable(tf.random.normal((2, 1)), name='weight1')
b1 = tf.Variable(tf.random.normal((1,)), name='bias1')

W2 = tf.Variable(tf.random.normal((2, 1)), name='weight2')
b2 = tf.Variable(tf.random.normal((1,)), name='bias2')

W3 = tf.Variable(tf.random.normal((2, 1)), name='weight3')
b3 = tf.Variable(tf.random.normal((1,)), name='bias3')

#Define Nueral Net
def neural_net(features):
    layer1 = tf.sigmoid(tf.matmul(features, W1) + b1)
    layer2 = tf.sigmoid(tf.matmul(features, W2) + b2)
    layer3 = tf.concat([layer1, layer2],-1)
    layer3 = tf.reshape(layer3, shape = [-1,2])
    hypothesis = tf.sigmoid(tf.matmul(layer3, W3) + b3)
    return hypothesis

#Cost function
def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

#Accuracy Function
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

#Gradient Descent
def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(neural_net(features),labels)
    return tape.gradient(loss_value, [W1, W2, W3, b1, b2, b3])

#Model fitting
EPOCHS = 50000

for step in range(EPOCHS):
    for features, labels  in dataset:
        features, labels = preprocess_data(features, labels)
        grads = grad(neural_net(features), features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W1, W2, W3, b1, b2, b3]))
        if step % 5000 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(neural_net(features),labels)))
x_data, y_data = preprocess_data(x_data, y_data)
test_acc = accuracy_fn(neural_net(x_data),y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))

>**class로 만든 코드**

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))
nb_classes = 10

class wide_deep_nn():
    def __init__(self, nb_classes):
        super(wide_deep_nn, self).__init__()

        self.W1 = tf.Variable(tf.random.normal((2, nb_classes)), name='weight1')
        self.b1 = tf.Variable(tf.random.normal((nb_classes,)), name='bias1')

        self.W2 = tf.Variable(tf.random.normal((nb_classes, nb_classes)), name='weight2')
        self.b2 = tf.Variable(tf.random.normal((nb_classes,)), name='bias2')

        self.W3 = tf.Variable(tf.random.normal((nb_classes, nb_classes)), name='weight3')
        self.b3 = tf.Variable(tf.random.normal((nb_classes,)), name='bias3')

        self.W4 = tf.Variable(tf.random.normal((nb_classes, 1)), name='weight4')
        self.b4 = tf.Variable(tf.random.normal((1,)), name='bias4')

        self.variables = [self.W1,self.b1,self.W2,self.b2,self.W3,self.b3,self.W4,self.b4]

    def preprocess_data(self, features, labels):
        features = tf.cast(features, tf.float32)
        labels = tf.cast(labels, tf.float32)
        return features, labels

    def deep_nn(self, features):
        layer1 = tf.sigmoid(tf.matmul(features, self.W1) + self.b1)
        layer2 = tf.sigmoid(tf.matmul(layer1, self.W2) + self.b2)
        layer3 = tf.sigmoid(tf.matmul(layer2, self.W3) + self.b3)
        hypothesis = tf.sigmoid(tf.matmul(layer3, self.W4) + self.b4)
        return hypothesis

    def loss_fn(self, hypothesis, features, labels):
        cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
        return cost

    def accuracy_fn(self, hypothesis, labels):
        predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
        return accuracy

    def grad(self, hypothesis, features, labels):
        with tf.GradientTape() as tape:
            loss_value = self.loss_fn(self.deep_nn(features),features,labels)
        return tape.gradient(loss_value,self.variables)

    def fit(self, dataset, EPOCHS=20000, verbose=500):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.01)
        for step in range(EPOCHS):
            for features, labels  in dataset:
                features, labels = self.preprocess_data(features, labels)
                grads = self.grad(self.deep_nn(features), features, labels)
                optimizer.apply_gradients(grads_and_vars=zip(grads, self.variables))
                if step % verbose == 0:
                    print("Iter: {}, Loss: {:.4f}".format(step, self.loss_fn(self.deep_nn(features),features,labels)))

    def test_model(self,x_data, y_data):
        x_data, y_data = self.preprocess_data(x_data, y_data)
        test_acc = self.accuracy_fn(self.deep_nn(x_data),y_data)
        print("Testset Accuracy: {:.4f}".format(test_acc))

In [ ]:
model = wide_deep_nn(nb_classes)
model.fit(dataset)
model.test_model(x_data, y_data)

# **Lab 09-2: TesorBoard (Neural Net for XOR)**

In [ ]:
!pip install tensorboard

In [ ]:
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

plt.scatter(x_data[0][0],x_data[0][1], c='red' , marker='^')
plt.scatter(x_data[3][0],x_data[3][1], c='red' , marker='^')
plt.scatter(x_data[1][0],x_data[1][1], c='blue' , marker='^')
plt.scatter(x_data[2][0],x_data[2][1], c='blue' , marker='^')

plt.xlabel("x1")
plt.ylabel("x2")
plt.show()
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

def preprocess_data(features, labels):
    features = tf.cast(features, tf.float32)
    labels = tf.cast(labels, tf.float32)
    return features, labels

>**TensorBoard**  
- Tensorboard를 이용해서 weight, bias, data, loss 값을 매 epoch 마다 저장하고 그래프를 그려준다

In [ ]:
log_path = "./logs/xor"
writer = tf.summary.create_file_writer(log_path)

In [ ]:
W1 = tf.Variable(tf.random.normal((2, 10)), name='weight1')
b1 = tf.Variable(tf.random.normal((10,)), name='bias1')

W2 = tf.Variable(tf.random.normal((10, 10)), name='weight2')
b2 = tf.Variable(tf.random.normal((10,)), name='bias2')

W3 = tf.Variable(tf.random.normal((10, 10)), name='weight3')
b3 = tf.Variable(tf.random.normal((10,)), name='bias3')

W4 = tf.Variable(tf.random.normal((10, 1)), name='weight4')
b4 = tf.Variable(tf.random.normal((1,)), name='bias4')

def neural_net(features, step):
    layer1 = tf.sigmoid(tf.matmul(features, W1) + b1)
    layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2)
    layer3 = tf.sigmoid(tf.matmul(layer2, W3) + b3)
    hypothesis = tf.sigmoid(tf.matmul(layer3, W4) + b4)

    with writer.as_default():
        tf.summary.histogram("weights1", W1, step=step)
        tf.summary.histogram("biases1", b1, step=step)
        tf.summary.histogram("layer1", layer1, step=step)

        tf.summary.histogram("weights2", W2, step=step)
        tf.summary.histogram("biases2", b2, step=step)
        tf.summary.histogram("layer2", layer2, step=step)

        tf.summary.histogram("weights3", W3, step=step)
        tf.summary.histogram("biases3", b3, step=step)
        tf.summary.histogram("layer3", layer3, step=step)

        tf.summary.histogram("weights4", W4, step=step)
        tf.summary.histogram("biases4", b4, step=step)
        tf.summary.histogram("hypothesis", hypothesis, step=step)
    return hypothesis

def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
    with writer.as_default():
        tf.summary.scalar('loss', cost, step=step)
    return cost

optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

def grad(hypothesis, features, labels, step):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(neural_net(features, step),labels)
    return tape.gradient(loss_value, [W1, W2, W3, W4, b1, b2, b3, b4])

In [ ]:
EPOCHS = 3000

for step in range(EPOCHS):
    for features, labels  in dataset:
        features, labels = preprocess_data(features, labels)
        grads = grad(neural_net(features, step), features, labels, step)
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W1, W2, W3, W4, b1, b2, b3, b4]))
        if step % 50 == 0:
            loss_value = loss_fn(neural_net(features, step),labels)
            print("Iter: {}, Loss: {:.4f}".format(step, loss_value))
x_data, y_data = preprocess_data(x_data, y_data)
test_acc = accuracy_fn(neural_net(x_data, step),y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))

>**Tensorboard에 있는 값을 jypyter notebook에서 load 하기**

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs/xor

#**Lab 10-1: Sigmoid 보다 ReLU가 더 좋아**

>**Problem of sigmoid**  
- Vanishing Gradient  
&rarr; layer가 많은 network에서 sigmoid 함수가 여러 개 있으면, 기울기가 0이 되는 지점이 많아져 gradient가 소실 되는 현상

> **ReLU Function**  
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcKMSix%2FbtrDAQM4SiV%2FmwPJt9tb1NuTyFiqv9YDM1%2Fimg.png">  
- $f(x) = max(0,x)$  
- tf.keras.activations에 여러가지 activation 함수가 많다  
- sigmoid, tanh, relu, elu, selu등 많다


>**Mnist Set 이용**

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

print(tf.__version__)

>**Load Mnist**  
- expand dimension 하는 이유는 [batch_size, height, width, channel]이기 때문에 하나 더 추가해야한다  
- [0 ~ 255]을 [0~1]로 normalization을 한다  
- 0 부터 9까지 숫자를 one hot coding을 진행한다  


In [ ]:
#Data Mining
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

#Normalization
def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

>**Performance Function**  
- Loss, Accuracy, gradient 정의

In [ ]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels,
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

>**Create network(function version)**  
- sigmoid와 relu 두 개를 정의하고 비교하자



In [ ]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def sigmoid() :
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [ ]:
def create_model_function(label_dim, mode) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    if mode == 1:
        for i in range(2) :
            model.add(dense(256, weight_init))
            model.add(sigmoid())
    elif mode == 2:
        for i in range(2) :
            model.add(dense(256, weight_init))
            model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model



>**Checkpoint Function**  
- 특정 epoch 마다 model의 weight와 bias를 저장해주는 function

In [ ]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

>**Experiments(parameters)**  


In [ ]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

>**Experiments (model)**

In [ ]:
""" Model """
network = create_model_function(label_dim, 2)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

>**Experiments(Eager mode)**

In [ ]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")

    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))

# test phase
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

&rarr; sigmoid는 0.9247  
&rarr; relu는 0.9356

# **Lab 10-2: Weight Initialization**

> **Xavier Initialization**  
- Local minimum에 빠질 수도 있고 saddle point에빠질 위험도 있다  
- 기존의 RandomNormalization Initialization에서는평균이 0 분산이 1인 초기 weight 값이었다  
- Xavier은 평균은 0이다<br><br>
- $Variance = \frac{2}{Channel\_in + Channel\_out}$
- ($Channel\_in$ : input으로 들어가는 채널의 개수, $Channel\_out : output으로 들어가는 채널의 개수$)  


> **He Initialization**  
- ReLU함수에 특화된 weight Initialization 기법  
- Xavier 분산에 2를 곱한 값을 취한다

>**Code**  
- weight_init = tf.keras.initializers. 뒷값을 바꿔주면 된다  
- 기존 : RandomNormal()  
- Xavier : glorot_uniform()  
- He : hr_uniform()  


In [ ]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# **Lab 10-3: Dropout**

> **Dropout**  
- underfitting과 overfitting을 막아주는 regularization  
- node 몇개를 끄고 학습시키는 방법  
- 어떤 node를 끄는가는 랜덤으로 설

In [ ]:
def dropout(rate) :
    return tf.keras.layers.Dropout(rate)

class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())
            ###이부분을 추가해준다
            self.model.add(dropout(rate=0.5))

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

# **Lab 10-4: Batch Normalization**

>**Internal Covariate Shift**  
- data가 layer를 지나가면서 distribution이 이상하게 생기는 현상  

>**Batch Normalization**  
- internal Covariate shift를 막기 위해 이용한다<br><br>
- $\bar{x} = \frac{x - \mu_{B}}{\sqrt{\sigma^{2}_{B} + \epsilon}}$  ($x$ : input, $\mu_{B}$ : Batch들의 평균, $\sqrt{\sigma^{2}_{B} + \epsilon}$ : Batch들의 분산<br><br>  
- $\hat{x} = \gamma\bar{x} + \beta$<br><br>
- 감마와 베타로 어떤 학습이 되는 파라미터들을 이용해서 $\hat{x}$을 다음 input으로 이용한다<br><br>
- 보통 layer -> norm -> activation 순으로 한다

In [ ]:
def batch_norm() :
    return tf.keras.layers.BatchNormalization()

In [ ]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            ###이 부분이 바뀐다
            self.model.add(batch_norm())
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x